# Post-Modeling Analysis

## Get performance tables

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import util
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim


In [2]:
# ALWAYS choose devise first.
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
import pickle
# performance_continuous
# performance_handcrafted

# performance_continuous is the result from CNN models
with open('outputs/performance_continuous_bw_resnet18.pickle', 'rb') as h:
    performance_continuous_bw = pickle.load(h) # pure cnn models with bw 

# with open('outputs/performance_continuous.pickle', 'rb') as h:
#     performance_continuous_bw = pickle.load(h) # pure cnn models with bw 

with open('outputs/performance_continuous_rgb_resnet18.pickle', 'rb') as h:
    performance_continuous_rgb = pickle.load(h) # pure cnn models with bw 

with open('outputs/performance_continuous_merge_resnet18.pickle', 'rb') as h:
    performance_continuous_merge = pickle.load(h) # pure cnn models with bw 
    
with open('outputs/performance_handcrafted.pickle', 'rb') as h:
    performance_handcrafted = pickle.load(h) # BE and NHTS models

with open('outputs/performance_cnn_combined.pickle', 'rb') as h:
    performance_cnn_combined = pickle.load(h) # CNN + BE + NHTS models

In [4]:
print(performance_continuous_bw)
print()
print(performance_handcrafted)
print()
print(performance_cnn_combined)

{'HHVEHCNT_mean_norm': {'mse_train_list': [12968.76671910286, 9803.879088163376, 9795.471370220184, 9716.708314418793, 9511.821076273918, 9420.36070227623, 9256.327441334724, 9344.197827577591, 8872.700893878937, 8167.845937609673, 7423.487824201584], 'mse_test_list': [2314.4653499126434, 2293.4097290039062, 2329.2478561401367, 2347.6718723773956, 2339.412635564804, 2312.281894683838, 2332.5907111167908, 2260.4303300380707, 2326.6711831092834, 2359.0803802013397, 2771.1448907852173], 'r_square_train_list': [-0.211983058210917, 0.08378833415979003, 0.08457406898917474, 0.09193479120060877, 0.11108232209955238, 0.11962965942034431, 0.1349592229440183, 0.12674738432038613, 0.17081065633525172, 0.23668216779023155, 0.3062454070891224], 'r_square_test_list': [0.09216375084142592, 0.10042270183855984, 0.08636539442738966, 0.07913867582652523, 0.08237831584495914, 0.09302019909423698, 0.08505417802762749, 0.11335868891556589, 0.08737608019104826, 0.07466375164942463, -0.08696627651839828]}, '

In [5]:
output_var_list=['HHVEHCNT_mean_norm', 'HHVEHCNT_P_CAP_mean_norm', 'TRPTRANS_1_mean_norm', 'TRPTRANS_2_mean_norm', 'TRPTRANS_3_mean_norm']
input_structure_continuous_list = ['ResNet18 (BW)', 'ResNet18 (RGB)', 'ResNet18 (Merge)']

input_structure_handcrafted_list = ['BE linear', 'BE quadratic', 'NHTS linear', 'NHTS quadratic', 'BE and NHTS linear', 
                                    'BE and NHTS quadratic']
input_structure_cnn_combined_list = ['CNN and NHTS linear', 'CNN BE NHTS linear']


In [6]:
train_r2 = {}
test_r2 = {}

for output_var in output_var_list:
    test_r2[output_var] = {}
    train_r2[output_var] = {}
    
    for input_structure_continuous in input_structure_continuous_list:
        if input_structure_continuous == 'ResNet18 (BW)':
            max_r_square_test = np.max(performance_continuous_bw[output_var]['r_square_test_list'])
            idx = performance_continuous_bw[output_var]['r_square_test_list'].index(max_r_square_test)        
            test_r2[output_var][input_structure_continuous]=np.max(performance_continuous_bw[output_var]['r_square_test_list'])
            train_r2[output_var][input_structure_continuous]=performance_continuous_bw[output_var]['r_square_train_list'][idx]

        elif input_structure_continuous == 'ResNet18 (RGB)':
            max_r_square_test = np.max(performance_continuous_rgb[output_var]['r_square_test_list'])
            idx = performance_continuous_rgb[output_var]['r_square_test_list'].index(max_r_square_test)        
            test_r2[output_var][input_structure_continuous]=np.max(performance_continuous_rgb[output_var]['r_square_test_list'])
            train_r2[output_var][input_structure_continuous]=performance_continuous_rgb[output_var]['r_square_train_list'][idx]
            
        elif input_structure_continuous == 'ResNet18 (Merge)':
            max_r_square_test = np.max(performance_continuous_merge[output_var]['r_square_test_list'])
            idx = performance_continuous_merge[output_var]['r_square_test_list'].index(max_r_square_test)        
            test_r2[output_var][input_structure_continuous]=np.max(performance_continuous_merge[output_var]['r_square_test_list'])
            train_r2[output_var][input_structure_continuous]=performance_continuous_merge[output_var]['r_square_train_list'][idx]

    for input_structure_handcrafted in input_structure_handcrafted_list:
        test_r2[output_var][input_structure_handcrafted]=performance_handcrafted[output_var][input_structure_handcrafted]['test_r2']
        train_r2[output_var][input_structure_handcrafted]=performance_handcrafted[output_var][input_structure_handcrafted]['train_r2']
        
    for input_structure_cnn_combined in input_structure_cnn_combined_list:
        test_r2[output_var][input_structure_cnn_combined]=performance_cnn_combined[output_var][input_structure_cnn_combined]['test_r2']
        train_r2[output_var][input_structure_cnn_combined]=performance_cnn_combined[output_var][input_structure_cnn_combined]['train_r2']


In [7]:
col_names = ['Automobiles', 'Automobile_per_capita', 'Walk/Cycling Mode Share', 'Automobile Mode Share', 'PT Mode Share']

In [8]:
test_r2_table = pd.DataFrame(test_r2)
test_r2_table.columns = col_names
test_r2_table

,Automobiles,Automobile_per_capita,Walk/Cycling Mode Share,Automobile Mode Share,PT Mode Share
ResNet18 (BW),0.113359,0.113529,0.305088,0.398991,0.167109
ResNet18 (RGB),0.136587,0.107040,0.307056,0.377197,0.193660
ResNet18 (Merge),0.138444,0.114491,0.314789,0.401089,0.191961
BE linear,0.084195,0.087349,0.178908,0.222090,0.110670
BE quadratic,0.094053,0.094257,0.223722,0.275076,0.126030
NHTS linear,0.341971,0.337261,0.318877,0.382529,0.169336
NHTS quadratic,0.365544,0.367325,0.317638,0.382133,0.147775
BE and NHTS linear,0.343357,0.338454,0.319964,0.385261,0.173725
BE and NHTS quadratic,0.366108,0.367583,0.323204,0.399684,0.173483
CNN and NHTS linear,0.340076,0.337915,0.348160,0.428899,0.209755


In [9]:
test_r2_table.to_latex(index=True)

'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &  Automobiles &  Automobile\\_per\\_capita &  Walk/Cycling Mode Share &  Automobile Mode Share &  PT Mode Share \\\\\n\\midrule\nResNet18 (BW)         &     0.113359 &               0.113529 &                 0.305088 &               0.398991 &       0.167109 \\\\\nResNet18 (RGB)        &     0.136587 &               0.107040 &                 0.307056 &               0.377197 &       0.193660 \\\\\nResNet18 (Merge)      &     0.138444 &               0.114491 &                 0.314789 &               0.401089 &       0.191961 \\\\\nBE linear             &     0.084195 &               0.087349 &                 0.178908 &               0.222090 &       0.110670 \\\\\nBE quadratic          &     0.094053 &               0.094257 &                 0.223722 &               0.275076 &       0.126030 \\\\\nNHTS linear           &     0.341971 &               0.337261 &                 0.318877 &               0.382529 &       0.169336 \\\\\nNHTS qu

In [10]:
train_r2_table = pd.DataFrame(train_r2)
train_r2_table.columns = col_names
train_r2_table

,Automobiles,Automobile_per_capita,Walk/Cycling Mode Share,Automobile Mode Share,PT Mode Share
ResNet18 (BW),0.126747,0.099748,0.249558,0.339322,0.197826
ResNet18 (RGB),0.135910,0.100295,0.274357,0.420456,0.227839
ResNet18 (Merge),0.150148,0.100705,0.287274,0.325066,0.253510
BE linear,0.077908,0.084790,0.155690,0.199716,0.119360
BE quadratic,0.092613,0.094117,0.182661,0.237287,0.146784
NHTS linear,0.328678,0.330419,0.279024,0.363458,0.238829
NHTS quadratic,0.363115,0.372650,0.312434,0.396319,0.276492
BE and NHTS linear,0.329871,0.331471,0.284095,0.369893,0.243042
BE and NHTS quadratic,0.370054,0.378981,0.326017,0.413437,0.298096
CNN and NHTS linear,0.341037,0.338289,0.317712,0.448159,0.335393


In [11]:
train_r2_table.to_latex(index=True)

'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &  Automobiles &  Automobile\\_per\\_capita &  Walk/Cycling Mode Share &  Automobile Mode Share &  PT Mode Share \\\\\n\\midrule\nResNet18 (BW)         &     0.126747 &               0.099748 &                 0.249558 &               0.339322 &       0.197826 \\\\\nResNet18 (RGB)        &     0.135910 &               0.100295 &                 0.274357 &               0.420456 &       0.227839 \\\\\nResNet18 (Merge)      &     0.150148 &               0.100705 &                 0.287274 &               0.325066 &       0.253510 \\\\\nBE linear             &     0.077908 &               0.084790 &                 0.155690 &               0.199716 &       0.119360 \\\\\nBE quadratic          &     0.092613 &               0.094117 &                 0.182661 &               0.237287 &       0.146784 \\\\\nNHTS linear           &     0.328678 &               0.330419 &                 0.279024 &               0.363458 &       0.238829 \\\\\nNHTS qu

In [12]:
for i in range(test_r2_table.shape[0]):
    for j in range(test_r2_table.shape[1]):
        test_r2_table.iloc[i,j] = str(np.around(test_r2_table.iloc[i,j], decimals=4)*100)[:4]+'%'
        
test_r2_table.to_csv("outputs/test_r2_table.csv")
test_r2_table

,Automobiles,Automobile_per_capita,Walk/Cycling Mode Share,Automobile Mode Share,PT Mode Share
ResNet18 (BW),11.3%,11.3%,30.5%,39.9%,16.7%
ResNet18 (RGB),13.6%,10.7%,30.7%,37.7%,19.3%
ResNet18 (Merge),13.8%,11.4%,31.4%,40.1%,19.2%
BE linear,8.42%,8.73%,17.8%,22.2%,11.0%
BE quadratic,9.41%,9.43%,22.3%,27.5%,12.6%
NHTS linear,34.2%,33.7%,31.8%,38.2%,16.9%
NHTS quadratic,36.5%,36.7%,31.7%,38.2%,14.7%
BE and NHTS linear,34.3%,33.8%,32.0%,38.5%,17.3%
BE and NHTS quadratic,36.6%,36.7%,32.3%,39.9%,17.3%
CNN and NHTS linear,34.0%,33.7%,34.8%,42.8%,20.9%


In [13]:
print(test_r2_table.to_latex(index=True))

\begin{tabular}{llllll}
\toprule
{} & Automobiles & Automobile\_per\_capita & Walk/Cycling Mode Share & Automobile Mode Share & PT Mode Share \\
\midrule
ResNet18 (BW)         &       11.3\% &                 11.3\% &                   30.5\% &                 39.9\% &         16.7\% \\
ResNet18 (RGB)        &       13.6\% &                 10.7\% &                   30.7\% &                 37.7\% &         19.3\% \\
ResNet18 (Merge)      &       13.8\% &                 11.4\% &                   31.4\% &                 40.1\% &         19.2\% \\
BE linear             &       8.42\% &                 8.73\% &                   17.8\% &                 22.2\% &         11.0\% \\
BE quadratic          &       9.41\% &                 9.43\% &                   22.3\% &                 27.5\% &         12.6\% \\
NHTS linear           &       34.2\% &                 33.7\% &                   31.8\% &                 38.2\% &         16.9\% \\
NHTS quadratic        &       36.5\% &    

In [14]:
for i in range(train_r2_table.shape[0]):
    for j in range(train_r2_table.shape[1]):
        train_r2_table.iloc[i,j] = str(np.around(train_r2_table.iloc[i,j], decimals=4)*100)[:4]+'%'
   
train_r2_table.to_csv("outputs/train_r2_table.csv")
train_r2_table

,Automobiles,Automobile_per_capita,Walk/Cycling Mode Share,Automobile Mode Share,PT Mode Share
ResNet18 (BW),12.6%,9.96%,24.9%,33.9%,19.7%
ResNet18 (RGB),13.5%,10.0%,27.4%,42.0%,22.7%
ResNet18 (Merge),15.0%,10.0%,28.7%,32.5%,25.3%
BE linear,7.79%,8.48%,15.5%,19.9%,11.9%
BE quadratic,9.26%,9.41%,18.2%,23.7%,14.6%
NHTS linear,32.8%,33.0%,27.9%,36.3%,23.8%
NHTS quadratic,36.3%,37.2%,31.2%,39.6%,27.6%
BE and NHTS linear,32.9%,33.1%,28.4%,36.9%,24.3%
BE and NHTS quadratic,37.0%,37.9%,32.6%,41.3%,29.8%
CNN and NHTS linear,34.1%,33.8%,31.7%,44.8%,33.5%


In [15]:
print(train_r2_table.to_latex(index=True))

\begin{tabular}{llllll}
\toprule
{} & Automobiles & Automobile\_per\_capita & Walk/Cycling Mode Share & Automobile Mode Share & PT Mode Share \\
\midrule
ResNet18 (BW)         &       12.6\% &                 9.96\% &                   24.9\% &                 33.9\% &         19.7\% \\
ResNet18 (RGB)        &       13.5\% &                 10.0\% &                   27.4\% &                 42.0\% &         22.7\% \\
ResNet18 (Merge)      &       15.0\% &                 10.0\% &                   28.7\% &                 32.5\% &         25.3\% \\
BE linear             &       7.79\% &                 8.48\% &                   15.5\% &                 19.9\% &         11.9\% \\
BE quadratic          &       9.26\% &                 9.41\% &                   18.2\% &                 23.7\% &         14.6\% \\
NHTS linear           &       32.8\% &                 33.0\% &                   27.9\% &                 36.3\% &         23.8\% \\
NHTS quadratic        &       36.3\% &    

In [16]:
test_r2_table.shape

(11, 5)

## Visualize prediction

#### Resnet prediction

In [ ]:
### 
size = 12000
image_array_ = np.load("data_process/image_array_bw_tract_large.npy", mmap_mode='r')
image_array = image_array_[:size,]
image_array_torch = torch.from_numpy(image_array.astype("float32"))
image_array_torch_norm = image_array_torch/255

### 
output_var = 'TRPTRANS_1_mean_norm' # walking and cycling mode share.
# output_var = 'TRPTRANS_2_mean_norm' # walking and cycling mode share.
# output_var = 'HHVEHCNT_mean_norm' # walking and cycling mode share.
image_type='bw'
df_ = pd.read_csv("data_process/df_merged_tract_large.csv")
df = df_.iloc[:size,]
y = df[output_var].values.astype("float32")
y_torch = torch.from_numpy(y)

In [ ]:
# initialize model
num_categories = 1
model = models.resnet18()
model.conv1 = nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_categories) # if output_type == continuous, then num_categories = 1.
# load the saved model
model_name = 'resnet18'
PATH = './models/'+model_name+'_'+output_var+'_'+image_type+'.pth'
# PATH = './models/'+model_name+'_'+output_var+'.pth'
model.load_state_dict(torch.load(PATH))

In [ ]:
model.to(device)

In [ ]:
# compute prediction. Note since our model's training and testing performances are similar, I combine them for analysis.
y_pred = []

all_ds = TensorDataset(image_array_torch_norm, y_torch)
batch_size = 100
all_dl = DataLoader(all_ds, batch_size, shuffle = False) # again. No shuffle for prediction!

for images,labels in all_dl:
    # 
    images=images.to(device)
    labels=labels.to(device)
    # 
    outputs = model(images)
    y_pred.append(outputs.view(-1).cpu().detach().numpy())

#     print(outputs.size())

In [ ]:
y_pred_np = np.array(y_pred).reshape(-1)

In [ ]:
np.corrcoef(y, y_pred_np) # Good. 

In [ ]:
np.corrcoef(y[(y != np.min(y)) & (y != np.max(y))], y_pred_np[(y != np.min(y)) & (y != np.max(y))])

In [ ]:
plt.scatter(y, y_pred_np, s = 0.1)
# Finding 1: Truncated on one side (walk/cycling share) - the zeros cannot be predicted well. 
# Finding 2: Truncated on two sides (Car mode share). The numbers on the two sides cannot be predicted well. 
# Finding 3: Some y (HH Vehicle) has clear concentration around integers. It means that the data preprocessing is still not perfect. 
# These findings should lead to further innovations in modeling. May not be in this paper. 

In [ ]:
plt.scatter(y[(y != np.min(y)) & (y != np.max(y))], y_pred_np[(y != np.min(y)) & (y != np.max(y))], s = 0.1) # Good. Around .4 correlation, similar to R2 performance

#### NHTS prediction